In [1]:
# Using Drive as storage and github for version controll.

from google.colab import drive, userdata
import os

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Setup Paths (Change to your actual repo name)
REPO_PATH = "/content/drive/MyDrive/ML/DL_With_Pytorch"
%cd {REPO_PATH}

# 3. Secure Auth
token = userdata.get('GH_TOKEN')
username = "barada02"
repo = "DL_With_Pytorch"
!git remote set-url origin https://{token}@github.com/{username}/{repo}.git

# 4. Identity
!git config --global user.email "Chandanbarada2@gmail.com"
!git config --global user.name "Kumar"

!git pull origin main
print("✅ Environment Ready!")

Mounted at /content/drive
/content/drive/MyDrive/ML/DL_With_Pytorch
From https://github.com/barada02/DL_With_Pytorch
 * branch            main       -> FETCH_HEAD
Already up to date.
✅ Environment Ready!


In [3]:
# Push notebook changes to GitHub
# IMPORTANT: Press Ctrl+S (Save) before running this!
!git add .
!git commit -m "Intro "
!git push origin main

[main c2027f1] Intro
 1 file changed, 1 insertion(+), 1 deletion(-)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 871 bytes | 145.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/barada02/DL_With_Pytorch.git
   23184d8..c2027f1  main -> main


# Tensor Shapes

Manipulating tensor shapes is the bread and butter of deep learning. In PyTorch, these operations allow you to reorganize data to fit the expected input of different layers (like moving from a Convolutional layer to a Linear one)

# The flatten
### Reshape and view

Both change the dimensions of a tensor without changing its data, but they handle memory differently.

* view(): The OG method. It is very fast because it creates a "view" of the original data without copying it. Requirement: The tensor must be contiguous in memory. If you’ve just transposed a tensor, view() might throw an error.

* reshape(): The more robust sibling. It tries to return a view if possible, but if the data isn't contiguous, it will silently copy the data to a new memory block.

Pro Tip: Use -1 as a dimension to let PyTorch automatically calculate the size for that slot based on the remaining dimensions.